### Importing required libraries

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense

### Loading data

In [2]:
npz1 = np.load('Audiobooks_training.npz')
npz2 = np.load('Audiobooks_validation.npz')
npz3 = np.load('Audiobooks_testing.npz')

In [3]:
# Storing as "float" because we have scaled inputs
train_inputs = npz1['inputs'].astype(np.float) 

# As target is 0 or 1
train_targets = npz1['targets'].astype(np.int) 

In [4]:
# Storing as "float" because we have scaled inputs
validation_inputs = npz2['inputs'].astype(np.float) 

# As target is 0 or 1
validation_targets = npz2['targets'].astype(np.int) 

In [5]:
# Storing as "float" because we have scaled inputs
test_inputs = npz3['inputs'].astype(np.float) 

# As target is 0 or 1
test_targets = npz3['targets'].astype(np.int) 

### Creating our model

In [6]:
input_layer_size = 10 # Number of nodes in input layer
output_layer_size = 100 # Number of nodes in each hidden layer of the model
hidden_layer_size = 2 # Number of nodes in output layer

In [7]:
model = Sequential()
model.add(Dense(hidden_layer_size, activation='relu'))
model.add(Dense(hidden_layer_size, activation='relu'))
model.add(Dense(output_layer_size, activation='softmax'))

In [10]:
# Setting different hyperparameters for SGD optimizer to be used in the model
sgd = tf.keras.optimizers.SGD(learning_rate = 0.1, momentum = 0.9, nesterov = True)

In [11]:
model.compile(optimizer = 'sgd', 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])

### Training the model

In [14]:
num_epochs = 50
batch_size = 25

In [16]:
model.fit(x = train_inputs, 
          y = train_targets,
          batch_size = batch_size,
          epochs = num_epochs,
          validation_data = (validation_inputs, validation_targets),
          verbose = 2,
          shuffle = True
         )

Train on 3355 samples, validate on 671 samples
Epoch 1/50
3355/3355 - 1s - loss: 4.2123 - accuracy: 0.4742 - val_loss: 3.7780 - val_accuracy: 0.5156
Epoch 2/50
3355/3355 - 0s - loss: 3.0739 - accuracy: 0.5049 - val_loss: 2.0140 - val_accuracy: 0.5142
Epoch 3/50
3355/3355 - 0s - loss: 1.3071 - accuracy: 0.5088 - val_loss: 0.9611 - val_accuracy: 0.5142
Epoch 4/50
3355/3355 - 0s - loss: 0.8732 - accuracy: 0.5222 - val_loss: 0.8173 - val_accuracy: 0.4993
Epoch 5/50
3355/3355 - 0s - loss: 0.7869 - accuracy: 0.5216 - val_loss: 0.7644 - val_accuracy: 0.5455
Epoch 6/50
3355/3355 - 0s - loss: 0.7487 - accuracy: 0.5565 - val_loss: 0.7359 - val_accuracy: 0.5514
Epoch 7/50
3355/3355 - 0s - loss: 0.7259 - accuracy: 0.5618 - val_loss: 0.7168 - val_accuracy: 0.5544
Epoch 8/50
3355/3355 - 1s - loss: 0.7096 - accuracy: 0.5815 - val_loss: 0.7092 - val_accuracy: 0.5604
Epoch 9/50
3355/3355 - 1s - loss: 0.6967 - accuracy: 0.5982 - val_loss: 0.6871 - val_accuracy: 0.6125
Epoch 10/50
3355/3355 - 0s - loss: 

Though our training loss consistenly decreases, our validation loss sometimes increase and then again decreases. <br> It means we have overfitted our model. Hence, we have to implement an early stopping mechanism.

### Implementing early stopping mechanism

In [19]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 3)

In [20]:
model.fit(x = train_inputs, 
          y = train_targets,
          batch_size = batch_size,
          epochs = num_epochs,
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose = 2,
          shuffle = True
         )

Train on 3355 samples, validate on 671 samples
Epoch 1/50
3355/3355 - 0s - loss: 0.2725 - accuracy: 0.9007 - val_loss: 0.2736 - val_accuracy: 0.9106
Epoch 2/50
3355/3355 - 1s - loss: 0.2713 - accuracy: 0.9049 - val_loss: 0.2755 - val_accuracy: 0.9136
Epoch 3/50
3355/3355 - 1s - loss: 0.2682 - accuracy: 0.9061 - val_loss: 0.2865 - val_accuracy: 0.8942
Epoch 4/50
3355/3355 - 1s - loss: 0.2703 - accuracy: 0.9049 - val_loss: 0.2699 - val_accuracy: 0.9106
Epoch 5/50
3355/3355 - 0s - loss: 0.2700 - accuracy: 0.9049 - val_loss: 0.2730 - val_accuracy: 0.9091
Epoch 6/50
3355/3355 - 0s - loss: 0.2680 - accuracy: 0.9043 - val_loss: 0.2676 - val_accuracy: 0.9091
Epoch 7/50
3355/3355 - 1s - loss: 0.2666 - accuracy: 0.9049 - val_loss: 0.2728 - val_accuracy: 0.9091
Epoch 8/50
3355/3355 - 1s - loss: 0.2671 - accuracy: 0.9061 - val_loss: 0.2694 - val_accuracy: 0.9106
Epoch 9/50
3355/3355 - 1s - loss: 0.2655 - accuracy: 0.9076 - val_loss: 0.2802 - val_accuracy: 0.9076


### Testing the model

In [23]:
test_loss, test_accuracy = model.evaluate(x = test_inputs, y = test_targets, verbose = 2)

448/1 - 0s - loss: 0.2607 - accuracy: 0.8817


In [30]:
print("Test Loss: {0:.2f}".format(test_loss))
print("Test Accuracy: {0:.2f}%".format(test_accuracy*100))

Test Loss: 0.33
Test Accuracy: 88.17%
